In [ ]:
import tensorflow
import keras

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/CatClassifier/train'

In [ ]:
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=15,)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='training',
    class_mode='binary',
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    subset='validation',
    class_mode='binary'
)


Found 20001 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# 추가 학습을 위해 미리 저장한 모델을 불러옴
additional_model = keras.models.load_model('/content/drive/MyDrive/CatClassifier/VggNet')

In [ ]:
additional_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [ ]:
additional_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, decay = 0.0005),
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/CatClassifier/VggNet',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False,
                             mode='auto',)

early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose = 1, mode= 'auto')

In [ ]:
num_train_samples = train_generator.n
num_train_batches = train_generator.batch_size
print(num_train_samples,' , ',num_train_batches)
train_size = num_train_samples // num_train_batches
print(train_size)

20001  ,  32
625


In [ ]:
num_val_samples = validation_generator.n
num_val_batches = validation_generator.batch_size
print(num_val_samples,' , ',num_val_batches)
val_size = num_val_samples // num_val_batches
print(val_size)

5000  ,  32
156


In [ ]:
hist = additional_model.fit(train_generator,
                 steps_per_epoch=train_size, 
                 epochs = 100 , 
                 validation_data=validation_generator ,
                 validation_steps=val_size ,
                 verbose=1,
                 callbacks=[checkpoint, early])

Epoch 1/100
625/625 [==============================] - 19482s 31s/step - loss: 0.6231 - accuracy: 0.6331 - val_loss: 0.6015 - val_accuracy: 0.6585

Epoch 00001: val_accuracy improved from -inf to 0.65845, saving model to /content/drive/MyDrive/CatClassifier/VggNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/VggNet/assets
Epoch 2/100
625/625 [==============================] - 405s 648ms/step - loss: 0.6111 - accuracy: 0.6478 - val_loss: 0.5866 - val_accuracy: 0.6837

Epoch 00002: val_accuracy improved from 0.65845 to 0.68369, saving model to /content/drive/MyDrive/CatClassifier/VggNet
INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/VggNet/assets
Epoch 3/100
625/625 [==============================] - 396s 633ms/step - loss: 0.5975 - accuracy: 0.6694 - val_loss: 0.5927 - val_accuracy: 0.6675

Epoch 00003: val_accuracy did not improve from 0.68369
Epoch 4/100
625/625 [==============================] - 395s 632ms/step - loss: 0.5915 - accu

In [ ]:
additional_model.evaluate(train_generator)

626/626 [==============================] - 295s 471ms/step - loss: 0.2788 - accuracy: 0.8791


[0.27880993485450745, 0.8790560364723206]

In [ ]:
additional_model.save('/content/drive/MyDrive/CatClassifier/VggNet/additional train (2)')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CatClassifier/VggNet/additional train (2)/assets
